In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_columns

20

### Creating the working set (FAO Data)

In [2]:
crop_harvest = pd.read_csv("./raw_data/fao_data/fao_harvest.csv")
crop_prod = pd.read_csv("./raw_data/fao_data/fao_prod.csv")
crop_yield = pd.read_csv("./raw_data/fao_data/fao_yield.csv")
change_temp = pd.read_csv("./raw_data/fao_data/fao_temp.csv")

In [3]:
crop_harvest = crop_harvest.loc[(crop_harvest["Flag"] == "A") | (crop_harvest["Flag"] == "E")]
crop_harvest = crop_harvest.reset_index(drop=True)
crop_harvest = crop_harvest.drop_duplicates()

crop_prod = crop_prod.loc[(crop_prod["Flag"] == "A") | (crop_prod["Flag"] == "E")]
crop_prod = crop_prod.reset_index(drop=True)
crop_prod = crop_prod.drop_duplicates()

crop_yield = crop_yield.loc[(crop_yield["Flag"] == "A") | (crop_yield["Flag"] == "E")]
crop_yield = crop_yield.reset_index(drop=True)
crop_yield = crop_yield.drop_duplicates()
crop_yield["Value"] = crop_yield["Value"].apply(lambda x: x / 10_000)

change_temp = change_temp.loc[(change_temp["Flag"] == "A") | (change_temp["Flag"] == "E")]
change_temp = change_temp.reset_index(drop=True)
change_temp = change_temp.drop_duplicates()
change_temp = change_temp[change_temp["Element"] == "Temperature change"]

In [4]:
crop_harvest = crop_harvest.rename(columns={"Value":"Harvest (ha)"})
crop_prod = crop_prod.rename(columns={"Value":"Production (tonnes)"})
crop_yield = crop_yield.rename(columns={"Value":"Yield (tonnes/ha)"})
change_temp = change_temp.rename(columns={"Value":"Change (C)"})

In [5]:
# Combines both dataframes and creates the final FAO dataframe
## Keep 5 columns, year item Harvest (ha) Production (tonnes) Yiel (tonnes/ha)
final_fao = crop_yield[["Year","Item", "Yield (tonnes/ha)"]] \
.merge(crop_harvest[["Year","Item", "Harvest (ha)"]], on=["Year", "Item"], how="inner")
final_fao = final_fao.merge(crop_prod[["Year","Item", "Production (tonnes)"]], on=["Year", "Item"], how="inner")
final_fao = final_fao.merge(change_temp[["Year", "Change (C)"]], on="Year", how="inner")
final_fao

,Year,Item,Yield (tonnes/ha),Harvest (ha),Production (tonnes),Change (C)
0,1991,Apples,19.9184,39200,780800.0,0.279
1,1991,Apricots,9.0000,100,900.0,0.279
2,1991,Asparagus,2.4751,6253,15477.0,0.279
3,1991,Barley,5.7166,2535384,14493757.0,0.279
4,1991,"Broad beans and horse beans, dry",3.5693,24281,86667.0,0.279
...,...,...,...,...,...,...
2140,1968,Sugar beet,42.7049,493591,21078736.0,0.204
2141,1968,Tomatoes,29.2348,1853,54172.0,0.204
2142,1968,"True hemp, raw or retted",0.6347,1875,1190.0,0.204
2143,1968,Unmanufactured tobacco,1.9388,7372,14293.0,0.204


In [6]:
# Overlook of data
final_fao[(final_fao["Year"] == 2021)].sample(10)# & final_data["Item"] == "Wheat"]#.iloc[0]

,Year,Item,Yield (tonnes/ha),Harvest (ha),Production (tonnes),Change (C)
1222,2021,Hop cones,2.2919,20900,47900.0,1.304
1219,2021,Cucumbers and gherkins,106.4912,2280,242800.0,1.304
1221,2021,Grapes,11.4335,100710,1151470.0,1.304
1213,2021,"Broad beans and horse beans, green",6.3243,370,2340.0,1.304
1244,2021,Spinach,20.2685,4320,87560.0,1.304
1250,2021,Wheat,7.3015,2939000,21459200.0,1.304
1247,2021,Sunflower seed,2.6031,38300,99700.0,1.304
1226,2021,Maize (corn),10.3608,430700,4462400.0,1.304
1220,2021,Currants,6.4648,2130,13770.0,1.304
1246,2021,Sugar beet,81.7645,390700,31945400.0,1.304


### Creating the working Subset (DWD Data)

In [7]:
precip = pd.read_csv("./raw_data/precipitation/observed_average_annual_precipitation.csv")
air_temp_mean = pd.read_csv("./raw_data/air_temperature/average_air_temperature.txt", sep=";", index_col=False)
frost_days = pd.read_csv("./raw_data/frost_days/regional_averages_tnas_year.txt", sep=";", skiprows=1)
hot_days = pd.read_csv("./raw_data/hot_days/regional_averages_txbs_year.txt", sep=";", skiprows=1)
ice_days = pd.read_csv("./raw_data/ice_days/regional_averages_txcs_year.txt", sep=";", skiprows=1)
summer_days = pd.read_csv("./raw_data/summer_days/regional_averages_txas_year.txt", sep=";", skiprows=1)#, index_col=False) 
sunshine_duration = pd.read_csv("./raw_data/sunshine_duration/regional_averages_sd_year.txt", sep=";", skiprows=1) # hours
tropical_nights = pd.read_csv("./raw_data/tropical_nights/regional_averages_tnes_year.txt", sep=";",skiprows=1)

In [8]:
# Changing hours to days
sunshine_duration["Deutschland"] = sunshine_duration["Deutschland"].apply(lambda x: (x / 24) / 365)

In [9]:
# creating sub dataframes for ease of use
de_pre = precip[["Category", "Annual Mean"]]
de_pre = de_pre.rename(columns={"Annual Mean":"Annual Mean Precipitation"})
de_temp = air_temp_mean[["Jahr", "Deutschland"]]
de_temp = de_temp.rename(columns={"Deutschland":"Air Temp Mean"})
de_frost = frost_days[["Jahr", "Deutschland"]]
de_frost = de_frost.rename(columns={"Deutschland":"Frost Days"})
de_hot = hot_days[["Jahr", "Deutschland"]]
de_hot = de_hot.rename(columns={"Deutschland":"Hot Days"})
de_ice = ice_days[["Jahr", "Deutschland"]]
de_ice = de_ice.rename(columns={"Deutschland":"Ice Days"})
de_sum = summer_days[["Jahr", "Deutschland"]]
de_sum = de_sum.rename(columns={"Deutschland":"Summer Days"})
de_sun = sunshine_duration[["Jahr", "Deutschland"]]
de_sun = de_sun.rename(columns={"Deutschland":"Sunshine Duration"})
de_trop = tropical_nights[["Jahr", "Deutschland"]]
de_trop = de_trop.rename(columns={"Deutschland":"Tropical Nights"})

In [10]:
## Merging all of the sub dataframes
final_dwd = de_pre.merge(de_temp, how="inner", left_on="Category", right_on="Jahr").merge(de_frost, how="inner").\
                    merge(de_hot, how="inner").merge(de_ice, how="inner").merge(de_sum, how="inner").merge(de_sun, how="inner").merge(de_trop, how="inner")

In [11]:
# master_dwd = master_dwd.set_index("Jahr", drop=True)
final_dwd = final_dwd.drop(columns=["Category"])
final_dwd = final_dwd.rename(columns={"Jahr":"Year"})

In [12]:
final_dwd

,Annual Mean Precipitation,Year,Air Temp Mean,Frost Days,Hot Days,Ice Days,Summer Days,Sunshine Duration,Tropical Nights
0,714.44,1951,8.72,85.57,3.02,9.81,27.53,0.195400,0.01
1,757.37,1952,7.94,109.43,7.91,25.70,30.70,0.179989,0.75
2,575.68,1953,8.94,94.54,5.08,23.40,32.96,0.203642,0.01
3,806.06,1954,7.70,96.70,2.53,33.25,17.18,0.178128,0.01
4,719.08,1955,7.54,117.22,0.93,31.93,21.40,0.186495,0.00
...,...,...,...,...,...,...,...,...,...
66,814.74,2017,9.58,76.17,6.80,16.63,39.01,0.182203,0.04
67,571.21,2018,10.45,77.35,20.37,14.64,74.72,0.230068,1.35
68,676.23,2019,10.28,69.74,16.97,7.69,52.29,0.209384,0.82
69,661.60,2020,10.43,62.51,11.39,3.71,45.37,0.216438,0.74


### Random Data

In [13]:
co2 = np.loadtxt(r"./raw_data/co2_mm_mlo.txt")
co2 = pd.DataFrame(data=co2, columns=["Year","Month","decimal date", "monthly average", "de-seasonalized", " number of days", "st.dev of days", "unc. of monthly mean"])
co2["Year"] = co2["Year"].astype(int)

In [14]:
co2.dtypes

Year                      int32
Month                   float64
decimal date            float64
monthly average         float64
de-seasonalized         float64
 number of days         float64
st.dev of days          float64
unc. of monthly mean    float64
dtype: object

In [15]:
co2 = co2[["Year", "monthly average"]]
co2 = co2.rename(columns={"monthly average":"Monthly Average"})
co2 = co2.groupby("Year", as_index=False).mean()

In [16]:
co2

,Year,Monthly Average
0,1958,315.232000
1,1959,315.980833
2,1960,316.910000
3,1961,317.644167
4,1962,318.454167
...,...,...
61,2019,411.654167
62,2020,414.238333
63,2021,416.450833
64,2022,418.564167


### Merging all datasets (FAO DWD RANDOM)

In [17]:
final_data = final_fao.merge(final_dwd, how="inner", left_on="Year", right_on="Year")
final_data = final_data.merge(co2, how="inner", left_on="Year", right_on="Year")

In [18]:
final_data = final_data.sort_values(by=["Year", "Item"], ignore_index=True)
final_data

,Year,Item,Yield (tonnes/ha),Harvest (ha),Production (tonnes),Change (C),Annual Mean Precipitation,Air Temp Mean,Frost Days,Hot Days,Ice Days,Summer Days,Sunshine Duration,Tropical Nights,Monthly Average
0,1961,Barley,2.3640,1552018,3669020.0,0.990,779.53,8.94,72.64,3.97,20.91,26.75,0.178333,0.00,317.644167
1,1961,"Beans, dry",1.8841,6737,12693.0,0.990,779.53,8.94,72.64,3.97,20.91,26.75,0.178333,0.00,317.644167
2,1961,"Broad beans and horse beans, dry",2.3147,34951,80900.0,0.990,779.53,8.94,72.64,3.97,20.91,26.75,0.178333,0.00,317.644167
3,1961,"Broad beans and horse beans, green",12.4613,1613,20100.0,0.990,779.53,8.94,72.64,3.97,20.91,26.75,0.178333,0.00,317.644167
4,1961,Cabbages,28.4699,30553,869841.0,0.990,779.53,8.94,72.64,3.97,20.91,26.75,0.178333,0.00,317.644167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,2021,Sugar beet,81.7645,390700,31945400.0,1.304,772.58,9.16,89.36,4.53,15.70,37.06,0.186210,0.29,416.450833
2141,2021,Sunflower seed,2.6031,38300,99700.0,1.304,772.58,9.16,89.36,4.53,15.70,37.06,0.186210,0.29,416.450833
2142,2021,Tomatoes,254.4250,400,101770.0,1.304,772.58,9.16,89.36,4.53,15.70,37.06,0.186210,0.29,416.450833
2143,2021,Triticale,5.8136,328300,1908600.0,1.304,772.58,9.16,89.36,4.53,15.70,37.06,0.186210,0.29,416.450833


## Saving the final dataframe into a csv

In [19]:
final_data.to_csv("dataset.csv")